In [1]:
import pandas as pd

In [2]:
hh = pd.read_parquet('data/hh_details.parquet')
hh.head()

,hh_size,employed_annual,nco_2015,nic_2008,max_income_from,hasincome_selfemp_agri,hasincome_wage_agri,hasincome_casual_agri,hh_type,hoh_religion,...,upper primary/middle,used_internet,days_outside_home,meals_daily,meals_school,meals_employer,meals_others,meals_payment,meals_home,mult
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,4,yes,112.0,47522.0,self-employment,NaN,NaN,NaN,1,Hinduism,...,0.0,4,0.0,12.0,0.0,0.0,3.0,10.0,347.0,57436
22300201,4,yes,112.0,73100.0,self-employment,NaN,NaN,NaN,1,Hinduism,...,2.0,3,0.0,12.0,0.0,0.0,0.0,10.0,350.0,27497
22300202,2,no,NaN,NaN,None,NaN,NaN,NaN,9,Hinduism,...,0.0,2,0.0,6.0,0.0,0.0,0.0,0.0,180.0,27497
22300203,2,no,NaN,NaN,None,NaN,NaN,NaN,9,Hinduism,...,1.0,0,0.0,6.0,0.0,0.0,6.0,0.0,174.0,27497
22300204,2,yes,231.0,85301.0,regular wage/salary earning,NaN,NaN,NaN,2,Christianity,...,0.0,2,0.0,6.0,0.0,0.0,3.0,2.0,175.0,27497


In [11]:
cols = ['rationcard_type', 'has_benefited_from_pmgky', 'meals_school', 'meals_others']

In [12]:
welfare = hh[cols]

In [14]:
proc = pd.read_parquet('data/hh-procurement.parquet')
proc.drop([c for c in proc if c.startswith('online_')], axis=1, inplace=True)
proc.head()

,used_ration_card,procured_rice,procured_wheat,procured_grain,procured_sugar,procured_pulses,procured_edible_oil,procured_other,performed_ceremony,meals_served_to_nonhh,...,has_mobile,has_bicycle,has_bike,has_car,has_truck,has_animalcart,has_fridge,has_washingmachine,has_ac,type_tv
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,no,False,False,False,False,False,False,False,no,5,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0
22300201,no,False,False,False,False,False,False,False,no,3,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN
22300202,no,False,False,False,False,False,False,False,no,2,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0
22300203,no,False,False,False,False,False,False,False,no,3,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0
22300204,no,False,False,False,False,False,False,False,no,3,...,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0


In [46]:
ration_cols = ['used_ration_card', 'procured_rice', 'procured_wheat', 'procured_pulses',
               'procured_grain', 'procured_sugar', 'procured_edible_oil', 'procured_other',
               'procured_kerosene_ration']
subsidies = ['received_subsidy_lpg', 'received_free_electricity', 'received_free_textbook', 'received_free_stationary', 'received_free_schoolbag',
             'received_free_oth_school', 'received_fee_waiver', 'received_free_laptop', 'received_free_tab', 'received_free_mobile',
             'received_free_bicycle', 'received_free_bike', 'received_free_clothing',
             'received_free_footwear', 'received_free_oth_dgq']
schemes = ['is_hhmem_pmjay', 'is_benefit_healthscheme']

In [48]:
wlf_proc = proc[ration_cols + subsidies + schemes]

In [49]:
basket_sources = pd.read_parquet('data/baskets-sources.parquet')
basket_sources.head()

item_code,71,72,140,141,142,143,144,145,146,148,...,301,302,309,320,321,322,323,324,325,329
hhid,,,,,,,,,,,,,,,,,,,,,
22300101,NaN,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
22300201,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22300202,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22300203,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
22300204,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
np.unique(basket_sources.values, return_counts=False)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., nan])

In [60]:
basket_sources.dropna(axis=1, how='all', inplace=True)  # Drop item codes that have all NaN sources

In [72]:
basket_sources.columns = [f'item_source_{i}' for i in basket_sources.columns]

In [73]:
len(welfare), len(wlf_proc), len(basket_sources)

(261953, 261953, 261953)

In [74]:
indicators = pd.concat([welfare, wlf_proc, basket_sources], axis=1, verify_integrity=True)